## ROMY Array - Map

### Importing

In [1]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import obspy
import pandas as pd

from obspy.clients.fdsn import Client, RoutingClient
from obspy.signal.array_analysis import array_transff_wavenumber
from obspy.imaging.cm import obspy_sequential
from obspy.geodetics import locations2degrees

from andbro__querrySeismoData import __querrySeismoData



In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/bay200/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'
elif os.uname().nodename == 'lin-ffb-01':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'

In [3]:
from functions.read_DEM import __read_DEM
from functions.array_latlon_to_xyz import __array_latlon_to_xyz

### Setting Variables

In [4]:
config = {}

config['klim'] = 50.
config['kstep'] = config['klim'] / 100.

config['tbeg'] = obspy.UTCDateTime("2024-03-13 12:00")
config['tend'] = obspy.UTCDateTime("2024-03-13 18:00")


config['seeds'] = [
                   'BW.GELB..HH*',
                   'BW.GRMB..HH*',
                   'BW.ALFT..HH*',
                   'BW.BIB..HH*',
                   'BW.TON..HH*',
                   'BW.FFB1..HH*',
                   'BW.FFB2..HH*',
                   'BW.FFB3..HH*',
                   'BW.DROMY..HH*',
                   'GR.FUR..HH*',
]

config['fmin'] = 0.001
config['fmax'] = 0.2


config['ref_station'] = 'BW.DROMY..HH*'

config['client'] = RoutingClient("eida-routing")

### Defining Methods

In [5]:
def __get_array_stations(seeds, tbeg, tend):

    route =  RoutingClient("eida-routing")

    for _i, seed in enumerate(seeds):

        net, sta, loc, cha = seed.split(".")


        inv =route.get_stations(
                                network=net,
                                station=sta,
                                channel=cha,
                                starttime=tbeg,
                                endtime=tend,
                                # latitude=ref_coords[0],
                                # longitude=ref_coords[1],
                                # minradius=0,
                                # maxradius=max_radius,
                            )
        if _i == 0:
            invs = inv
        else:
            invs += inv

    # inv_check.plot(projection="local");

    codes, lon, lat, elev = [], [], [], []
    for h, network in enumerate(invs):
        for i, station in enumerate(network):

            codes.append(f'{invs[h].code}.{station.code}')
            lon.append(station.longitude)
            lat.append(station.latitude)
            elev.append(station.elevation/1000.)

    # create dataframe
    frame = pd.DataFrame({'codes': codes, 'lon': lon, 'lat': lat, 'elev': elev})

    # sort the dataframe
    frame.sort_values(by="codes", inplace=True)

    return frame

In [6]:
array_stations = __get_array_stations(config['seeds'], config['tbeg'], config['tend'])

In [7]:
array_stations

,codes,lon,lat,elev
2,BW.ALFT,11.279500,48.142334,0.593
3,BW.BIB,11.247300,48.152200,0.599
8,BW.DROMY,11.275476,48.162941,0.571
5,BW.FFB1,11.275065,48.162904,0.572
6,BW.FFB2,11.273684,48.164357,0.577
7,BW.FFB3,11.276289,48.164883,0.562
0,BW.GELB,11.251400,48.162900,0.628
1,BW.GRMB,11.263500,48.140600,0.656
4,BW.TON,11.288809,48.173897,0.564
9,GR.FUR,11.276800,48.163900,0.565


### Add xyz and UTM

In [9]:
array_stations = __array_latlon_to_xyz(array_stations, ref_station="BW.FFB1..BHZ")

In [10]:
array_stations

,codes,lon,lat,elev,x_m,y_m,z_m,utm_n,utm_e,utm_zone,utm_letter
2,BW.ALFT,11.279500,48.142334,0.593,397.55,-2276.36,21.0,5.334633e+06,669567.346291,32.0,U
3,BW.BIB,11.247300,48.152200,0.599,-2029.66,-1250.37,27.0,5.335659e+06,667140.137071,32.0,U
8,BW.DROMY,11.275476,48.162941,0.571,30.44,5.02,-1.0,5.336915e+06,669200.235012,32.0,U
5,BW.FFB1,11.275065,48.162904,0.572,0.00,0.00,0.0,5.336910e+06,669169.797278,32.0,U
6,BW.FFB2,11.273684,48.164357,0.577,-107.46,158.45,5.0,5.337068e+06,669062.337441,32.0,U
7,BW.FFB3,11.276289,48.164883,0.562,84.50,222.64,-10.0,5.337132e+06,669254.292671,32.0,U
0,BW.GELB,11.251400,48.162900,0.628,-1759.57,-52.26,56.0,5.336857e+06,667410.222437,32.0,U
1,BW.GRMB,11.263500,48.140600,0.656,-786.92,-2504.24,84.0,5.334405e+06,668382.879818,32.0,U
4,BW.TON,11.288809,48.173897,0.564,985.54,1252.10,-8.0,5.338162e+06,670155.332442,32.0,U
9,GR.FUR,11.276800,48.163900,0.565,125.73,114.51,-7.0,5.337024e+06,669295.522351,32.0,U


## Compute station distance

In [ ]:
from functions.calculate_distance_of_stations import __calculate_distance_of_stations

In [ ]:
distances = __calculate_distance_of_stations(array_stations, output="km", ref_station=config['ref_station'])
distances

In [ ]:
cross_distances = __calculate_distance_of_stations(array_stations, output="km", ref_station=None)
cross_distances

### Calulcate Distances, Plot Array Geometry and Response

In [ ]:
def __plot_array_geometry_and_response(array, transff, klim, kstep, names=None, unit=None):

    import numpy as np

    # calulcate geometrical center of array
    array_center = [np.median(array[:, 0]), np.median(array[:, 1])]

    # find geometrical limits of array
    lon_min, lon_max = np.min(array[:, 0]), np.max(array[:, 0])
    lat_min, lat_max = np.min(array[:, 1]), np.max(array[:, 1])

    # define bounds
    kxmin, kxmax, kymin, kymax = -klim, klim, -klim, klim


    # _________________________________________________________
    #
    fig, axes = plt.subplots(1,2, figsize=(16,6))

    fs=12

    if unit == "km":
        xdata = (array[:, 0]-array_center[0])*111.
        ydata = (array[:, 1]-array_center[1])*111. * np.cos(array[:,1]/180.*np.pi)
        x0, y0 = array_center[0]*111., array_center[1]*111. * np.cos(array[1]/180.*np.pi)

    elif unit == "deg":
        xdata = array[:, 0]
        ydata = array[:, 1]
        x0, y0 = array_center[0], array_center[1]

    # ________________________________________________
    axes[0].scatter(xdata, ydata, s=35, color='black', marker='v', zorder=3)
    axes[0].scatter(x0, y0, color='red', marker='o', s=25, zorder=2)

    if unit == "km" or unit is None:
        axes[0].set_xlabel('X (km)', fontsize=fs)
        axes[0].set_ylabel('Y (km)', fontsize=fs)

        axes[0].set_ylim([-max(abs(ydata))*0.99, max(abs(ydata))*1.01])
        axes[0].set_xlim([-max(abs(xdata))*0.99, max(abs(xdata))*1.01])

    elif unit == "deg":
        axes[0].set_xlabel('Longitude (°)', fontsize=fs)
        axes[0].set_ylabel('Latitude (°)', fontsize=fs)

        axes[0].set_ylim([min(abs(ydata))*0.9999, max(abs(ydata))*1.0001])
        axes[0].set_xlim([min(abs(xdata))*0.999, max(abs(xdata))*1.001])

    axes[0].set_title('Array Stations', fontsize=fs+2)

    for x, y, s in zip(xdata, ydata, list(names)):
        axes[0].text(x, y, s, ha="right", va="top")

    # ________________________________________________
    ax2 = axes[1].pcolor(np.arange(kxmin, kxmax + kstep * 1.1, kstep) - kstep / 2.,
                         np.arange(kymin, kymax + kstep * 1.1, kstep) - kstep / 2.,
                         transff.T, cmap=obspy_sequential,
                        )

    plt.colorbar(ax2, ax=axes[:])
    ax2.set_clim(0., 1.)

    axes[1].set_xlim(kxmin, kxmax)
    axes[1].set_ylim(kymin, kymax)
    axes[1].set_xlabel('wave number (m$^{-1}$)', fontsize=fs)
    axes[1].set_ylabel('wave number (m$^{-1}$)', fontsize=fs)
    axes[1].set_title('Array Response Function', fontsize=fs+2)

    plt.show();

In [ ]:
# get maximal distance
dist_max = max(distances.values())

lambda_min = 1*dist_max # in km

config['k_max'] = 2*np.pi/lambda_min
config['k_step'] = config['k_max'] / 100.

coordinates = np.column_stack([array_stations.lon, array_stations.lat, array_stations.elev])

# compute transfer function as a function of wavenumber difference
transff = array_transff_wavenumber(coordinates,
                                   config['k_max'],
                                   config['k_step'],
                                   coordsys='lonlat'
                                   )


__plot_array_geometry_and_response(coordinates,
                                   transff,
                                   config['k_max'],
                                   config['k_step'],
                                   array_stations.codes,
                                   unit="deg",
                                  )
